# INCHAPE USED CARS: Sentiment Analysis for Customer Reviews - Data Collection

- https://www.trustpilot.com/review/inchcape.co.uk

## Webscraping using Selenium

In [1]:
url = 'https://www.trustpilot.com/review/inchcape.co.uk?'

In [2]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm #progress bars

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

In [3]:
%%time
browser = webdriver.Chrome('/Users/liannelelis/Downloads/chromedriver113')
browser.get(url)
browser.maximize_window()

<timed exec>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


CPU times: user 15.8 ms, sys: 17.6 ms, total: 33.3 ms
Wall time: 1min 19s


In [4]:
def clickNextPage():
    try:
        nextPage = EC.element_to_be_clickable((By.NAME, "pagination-button-next"))
        WebDriverWait(browser, 5).until(nextPage)

        time.sleep(2)  # Add a small delay to ensure the page is fully loaded
        nextPagebutton = browser.find_element(By.NAME, "pagination-button-next")
        browser.execute_script("arguments[0].click();", nextPagebutton)


    except TimeoutException:
        print("Timed out waiting for page to load.")

#### Get Reviews Updated

In [5]:
def getReviews(reviewBox):
    for box in reviewBox:
        retry_count = 0
        appended_data_count = 0
        while retry_count < MAX_RETRIES:
            try:
                content = box.find_elements(By.TAG_NAME, 'p')
                header = box.find_element(By.TAG_NAME, 'h2')
                headerText = header.text
                star = box.find_element(By.TAG_NAME, 'img')
                rate = star.get_attribute('alt')
                starRating.append(rate[6])

                appended_data_count = len(reviews)  # Keep track of the current appended data count

                for detail in content:
                    try:
                        text = detail.text
                        if 'Date of experience:' in text and len(content) == 1:
                            dateExp.append(text[20:])
                            reviews.append(headerText)
                        elif 'Date of experience:' in text:
                            dateExp.append(text[20:])
                        else:
                            reviews.append(text)
                    except StaleElementReferenceException:
                        print("StaleElementReferenceException occurred. Refreshing the page.")
                        browser.refresh()
                        wait_for_page_to_load()
                        print("Continuing from the same iteration...")
                        # Remove the appended data from the current iteration box
                        del starRating[-1]
                        del dateExp[appended_data_count:]
                        del reviews[appended_data_count:]
                        break  # Break the inner loop and continue from the same iteration
                    except NoSuchElementException:
                        print("NoSuchElementException occurred. Refreshing the page.")
                        browser.refresh()
                        wait_for_page_to_load()
                        print("Continuing from the same iteration...")
                        # Remove the appended data from the current iteration box
                        del starRating[-1]
                        del dateExp[appended_data_count:]
                        del reviews[appended_data_count:]
                        break  # Break the inner loop and continue from the same iteration

                break  # Break the outer loop if no exceptions occurred

            except NoSuchElementException:
                retry_count += 1
                print(f"NoSuchElementException occurred. Retrying... ({retry_count}/{MAX_RETRIES})")
                time.sleep(1)  # Wait for 1 second before retrying

        if retry_count == MAX_RETRIES:
            print("Max retries reached for detail content. Moving to the next detail.")

#### Wait for the page to fully load

In [6]:
def wait_for_page_to_load():
    try:
        WebDriverWait(browser, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__next"]/div/div/div/main/div/div[4]/section/div/article/div/section'))
        )
        time.sleep(2)
    except TimeoutException:
        print("Timed out waiting for page to load.")

#### Main Execution

In [7]:
reviews = []
starRating = []
dateExp = []
total_reviews = 0
total_dateExp = 0
total_starRating = 0

MAX_RETRIES = 3

pbar = tqdm(total=300, desc=f'Pages (Reviews: {total_reviews}, Star: {total_starRating}, Date: {total_dateExp})')

for i in range(300):   
    
    wait_for_page_to_load()
    
    reviewBox = browser.find_elements(By.XPATH, '//*[@id="__next"]/div/div/div/main/div/div[4]/section/div/article/div/section')
    
    try:
        getReviews(reviewBox)
    except StaleElementReferenceException:
        print("StaleElementReferenceException occurred while finding review content boxes.")
    except NoSuchElementException:
        print("NoSuchElementException occurred while finding review content boxes.")
    
    try:
        clickNextPage()
    except StaleElementReferenceException:
        print("StaleElementReferenceException occurred while clicking the next page.")
    except NoSuchElementException:
        print("NoSuchElementException occurred while clicking the next page.")
    
    wait_for_page_to_load()
    #wait_for_page_to_load()
    
    total_reviews = len(reviews)
    total_dateExp = len(dateExp)
    total_starRating = len(starRating)
    
    pbar.set_description(f'Pages{1+i} (Reviews: {total_reviews}, Star: {total_starRating}, Date: {total_dateExp})')
    pbar.update(1)

pbar.close()
    
    #print(f'{len(reviews)}, {len(dateExp)}')

Pages8 (Reviews: 160, Star: 160, Date: 160):   3%| | 8/300 [01:29<49:15, 10.12s/

StaleElementReferenceException occurred. Retrying... (1/3)
StaleElementReferenceException occurred. Retrying... (2/3)
StaleElementReferenceException occurred. Retrying... (3/3)
Max retries reached for detail content. Moving to the next detail.
StaleElementReferenceException occurred. Retrying... (1/3)
StaleElementReferenceException occurred. Retrying... (2/3)
StaleElementReferenceException occurred. Retrying... (3/3)
Max retries reached for detail content. Moving to the next detail.
StaleElementReferenceException occurred. Retrying... (1/3)
StaleElementReferenceException occurred. Retrying... (2/3)
StaleElementReferenceException occurred. Retrying... (3/3)
Max retries reached for detail content. Moving to the next detail.
StaleElementReferenceException occurred. Retrying... (1/3)
StaleElementReferenceException occurred. Retrying... (2/3)
StaleElementReferenceException occurred. Retrying... (3/3)
Max retries reached for detail content. Moving to the next detail.
StaleElementReferenceExc

KeyboardInterrupt: 

In [ ]:
pbar.close()

In [ ]:
print(browser.current_url)
browser.quit()

In [ ]:
url = browser.current_url
url

In [ ]:
dups = [x for x in reviews if reviews.count(x) >=2]
dups

#### Saving it into dataframe

In [ ]:
inchcape_reviews = pd.DataFrame({'Date of Exp': dateExp,
                                 'Star Rating': starRating,
                                 'Reviews': reviews
                                })

# converting the string to datetime format
inchcape_reviews['Date of Exp'] = pd.to_datetime(inchcape_reviews['Date of Exp'], format='%B %d, %Y')
# converting the string to integer format
inchcape_reviews['Star Rating'] = inchcape_reviews['Star Rating'].astype(int)

In [ ]:
inchcape_reviews.to_csv('inchcapeReviews5.csv',header=True, index = False)

In [ ]:
inchcape_reviews.info()

In [ ]:
len(dateExp)

###  scratch

In [ ]:
reviewBox

In [ ]:
star = reviewBox[4].find_element(By.TAG_NAME, 'img')
rate = star.get_attribute('alt')
rate[6]

In [ ]:
reviewBox[4]

In [ ]:
header = reviewBox[0].find_element(By.TAG_NAME, 'h2')
header.text